# The objective nature of Python's exceptions
The objective nature of Python's exceptions makes them very flexible, however, before diving into that lets cover a few additional syntactival and semantic aspects of `Try-Except`.

## `else`
The `else` keyword can also be used with `try-except`. In this usecase the `else` signifies *if there are no exceptions thrown*. So, if your `try` block executes without exception, the `else` branch will be executed also.

In [4]:
def division_func(n):
    try:
        n = 1 / n
    except ZeroDivisionError:
        print("Uh oh, looks like you tried to divide by zero.")
        return
    else:
        print("Wahoo, that workd. Here's your result:")
        return n
    
print(division_func(1))
print(division_func(0))

Wahoo, that workd. Here's your result:
1.0
Uh oh, looks like you tried to divide by zero.
None


## `finally`
The `finally` keyword can be used in combination with `else` or independently and is always executed regardless of whether the `try` or `except` block was executed.

In [25]:
def concat_func(n):
    try:
        n = "Bla bla " + n
    except TypeError:
        print("Nuh uh. Looks like you're trying to mix strings and integers there, hoss.")
    else:
        print("Success!")
    finally:
        print(f"Here you go: {n}")
        return "DONE"
        
print(concat_func(3))
print(concat_func("bla"))

Nuh uh. Looks like you're trying to mix strings and integers there, hoss.
Here you go: 3
DONE
Success!
Here you go: Bla bla bla
DONE


## `as`
Not only are exceptions classes but when an exception is raised an object of that class is instantiated. You can grab the instance for further evaluation by using the `as` keyword.

In [45]:
try:
    n = 1 / 0
except Exception as e:
    print(e)
    print(e.__str__())


division by zero
division by zero


## Exception classes
Here's a nice function from [edube](https://edube.org/) showing the various exception classes:

In [37]:
def print_exception_tree(thisclass, nest = 0):
    if nest > 1:
        print("   |" * (nest - 1), end="")
    if nest > 0:
        print("   +---", end="")

    print(thisclass.__name__)

    for subclass in thisclass.__subclasses__():
        print_exception_tree(subclass, nest + 1)


print_exception_tree(BaseException)


BaseException
   +---Exception
   |   +---TypeError
   |   |   +---MultipartConversionError
   |   |   +---FloatOperation
   |   +---StopAsyncIteration
   |   +---StopIteration
   |   +---ImportError
   |   |   +---ModuleNotFoundError
   |   |   +---ZipImportError
   |   +---OSError
   |   |   +---ConnectionError
   |   |   |   +---BrokenPipeError
   |   |   |   +---ConnectionAbortedError
   |   |   |   +---ConnectionRefusedError
   |   |   |   +---ConnectionResetError
   |   |   |   |   +---RemoteDisconnected
   |   |   +---BlockingIOError
   |   |   +---ChildProcessError
   |   |   +---FileExistsError
   |   |   +---FileNotFoundError
   |   |   +---IsADirectoryError
   |   |   +---NotADirectoryError
   |   |   +---InterruptedError
   |   |   |   +---InterruptedSystemCall
   |   |   +---PermissionError
   |   |   +---ProcessLookupError
   |   |   +---TimeoutError
   |   |   +---UnsupportedOperation
   |   |   +---ItimerError
   |   |   +---Error
   |   |   |   +---SameFileError
   |  

## The `args` property
Python's `BaseException` class (the topmost in the exception class hierarchy) introduces a property named `args`. `args` is a tuple designed to hold all arguments passed to the class constructor (it's empty if the constructor has been invoked without arguments).

In [5]:
def print_args(args):
    if len(args) == 0:
        print("No arguments were passed")
    elif len(args) == 1:
        print(args[0])
    else:
        print(str(args))
        
try:
    raise Exception
except Exception as e:
    print_args(e.args)
try:
    raise Exception("my","arguments")
except Exception as e:
    print_args(e.args)

No arguments were passed
('my', 'arguments')


## Defining your own exceptions
If needed you can create your own exceptions derived from Python's (e.g. an even more specific one derived from one of the more concrete exceptions or derived from one of the more abstract such as `Exception` if you're looking to create your own independent family of exceptions).

Lets say we've created a maths game where users are not permitted to divide by 2 and we want an exception specifically for that case.

In [14]:
import random

class DivByTwoError(ZeroDivisionError):
    message="You're not allowed do that."


def play_the_game(state):
    while state:
        userchoice = int(input("Please choose a number to divide by? "))
        if userchoice == 2:
            try:
                raise DivByTwoError
            except DivByTwoError as e:
                print(e.message)
                
        else:
            print(random.randint(1,100) / userchoice)
            state = False
            
play_the_game(True)

Please choose a number to divide by? 2
You're not allowed do that.
Please choose a number to divide by? 1
92.0


Or lets imagine we've created a bunch of musical instrument classes and want specific errors to handle issues we might run into.

In [30]:
class StringError(Exception): # base string error derived from Exception class
    def __init__(self, message, instrument):
        Exception.__init__(self, message)
        self.instrument = instrument
        
class BrokenString(StringError): # more specific error based on StringError
    def __init__(self, message, instrument, string):
        StringError.__init__(self, message, instrument)
        self.string = string
        self.message = f"string {self.string} on {self.instrument} is broken"
        
class DetunedString(StringError): # another specific error based on StringError
    def __init__(self, message, instrument, string, target_note, actual_note):
        StringError.__init__(self, message, instrument)
        self.string = string
        self.target_note = target_note
        self.actual_note = actual_note
        self.message = f"string {self.string} on {self.instrument} is tuned to {self.actual_note} when it should be {self.target_note}"
        
def play_music(instruments):
    for instrument in instruments:
        if instrument["status"] == "playing":
            print(f"{instrument['name']} is {instrument['status']}")
        elif instrument["status"] == "brokenstring":
            try:
                raise BrokenString(instrument["status"],instrument["name"],instrument["problem_string"])
            except BrokenString as e:
                print(e.message)
        elif instrument["status"] == "detuned":
            try:
                raise DetunedString(instrument["status"], 
                                    instrument["name"], 
                                    instrument["problem_string"], 
                                    instrument["target_note"], 
                                    instrument["actual_note"])
            except DetunedString as e:
                print(e.message)

lead_guitar = {
    "name": "lead guitar",
    "status":"playing",
    "problem_string": None
         }
bass = {
    "name": "bass",
    "status": "brokenstring",
    "problem_string": 4
}

rhythm_guitar = {
    "name": "rhythm guitar",
    "status": "detuned",
    "problem_string": 6,
    "target_note": "E",
    "actual_note": "D"
}

play_music([lead_guitar,bass,rhythm_guitar])

lead guitar is playing
string 4 on bass is broken
string 6 on rhythm guitar is tuned to D when it should be E
